In [ ]:
#  install several Python packages,
!pip install sentence_transformers
!pip -q install langchain tiktoken chromadb pypdf transformers
!pip -q install accelerate bitsandbytes sentencepiece Xformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 69.6 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.1/77.1 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 86.7 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 45.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 41.3 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 31.8 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.8/163.8 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 770.4/770.4 kB 26.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 65.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━

# LangChain multi-doc retriever with ChromaDB

In [ ]:

from langchain.vectorstores import Chroma  # Imports the Chroma module for vector operations.

from langchain.text_splitter import RecursiveCharacterTextSplitter  # Imports a method for recursive character-level text splitting.

from langchain.chains import RetrievalQA  # Imports a module for retrieving answers in a QA setup.

from langchain.document_loaders import TextLoader  # Imports a module for loading text data.

from langchain.document_loaders import DirectoryLoader  # Imports a module to load documents from a directory.


In [ ]:
# data source, https://themarketswatch.com/financial/top-stocks-to-buy/?gclid=CjwKCAjwyqWkBhBMEiwAp2yUFpSLJZSVwVHJR3co5yVHVLeBPqNNcU3VMrFJjniP3jtuupP6kHzDYxoC8dkQAvD_BwE
import zipfile
!wget https://github.com/Shafi2016/Youtube/raw/main/stock_market_june_2023.zip -O stock_market_june_2023.zip  # Downloads the zip file from the specified URL.

with zipfile.ZipFile('stock_market_june_2023.zip', 'r') as zip_ref:
    zip_ref.extractall()


--2023-06-17 18:25:26--  https://github.com/Shafi2016/Youtube/raw/main/stock_market_june_2023.zip
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/Shafi2016/Youtube/main/stock_market_june_2023.zip [following]
--2023-06-17 18:25:26--  https://raw.githubusercontent.com/Shafi2016/Youtube/main/stock_market_june_2023.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10638 (10K) [application/zip]
Saving to: ‘stock_market_june_2023.zip’

stock_market_june_2 100%[===================>]  10.39K  --.-KB/s    in 0s      

2023-06-17 18:25:26 (109 MB/s) - ‘stock_market_june_2023.zip’ saved [10638/10638]



In [ ]:

# Initializes a DirectoryLoader to load text files from the specified directory.
loader = DirectoryLoader('./stock_market_june_2023/', glob="./*.txt", loader_cls=TextLoader)

# Loads all the documents present in the directory.
documents = loader.load()

# The following line is commented out, but if used, it would load only the first 5 documents.
# documents = loader.load()[:5]


In [ ]:

# Initializes a RecursiveCharacterTextSplitter with a specified chunk size and overlap.
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)

# Splits the loaded documents into chunks of text using the defined text_splitter.
texts = text_splitter.split_documents(documents)


In [ ]:
len(texts)

49

In [ ]:
texts[0]

Document(page_content='Arbor Metals Corp.\nArbor Metals (TSX-V: ABR, FWB: 432, OTC: ABRMF) is a mineral exploration company that finds itself in the media spotlight as \nCanada continues to attract massive investments across the lithium and electric vehicle (EV) sectors. \nMajor media channels like Yahoo Finance, Financial Times, and CNN Business have recently focused on Arbor Metals Corp.’s strategic \nlocation in the James Bay region of Quebec.', metadata={'source': 'stock_market_june_2023/Arbor Metals Corp.txt'})

# vicuna-13B-1.1-HF

In [ ]:
import torch
import transformers
from transformers import LlamaTokenizer, LlamaForCausalLM, GenerationConfig, pipeline

tokenizer = LlamaTokenizer.from_pretrained("TheBloke/vicuna-13B-1.1-HF")

# Initializes a pretrained LlamaForCausalLM model from the specified path.
# The model is loaded in 8-bit precision for memory efficiency and set to use half-precision floating points (float16).
# The device_map argument set to 'auto' allows the model to be loaded on the best available device (CPU/GPU).
# The low_cpu_mem_usage argument set to True reduces the memory usage on the CPU.
model = LlamaForCausalLM.from_pretrained("TheBloke/vicuna-13B-1.1-HF",
                                         #load_in_8bit=True,
                                         device_map='auto',
                                         torch_dtype=torch.float16,
                                         low_cpu_mem_usage=True
                                         )


The model weights are not tied. Please use the `tie_weights` method before using the `infer_auto_device` function.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

# HuggingFace Embeddings

In [ ]:

from langchain.llms import HuggingFacePipeline
from langchain.embeddings import HuggingFaceEmbeddings
# Moving model to GPU if available
#model = model.to(device)

# Initializes a text generation pipeline with the pretrained model and tokenizer.
# The maximum length of the generated text is set to 2048 tokens.
# The temperature argument controls the randomness of predictions (0.7 allows for some randomness but still fairly focused).
# The top_p argument controls nucleus sampling, with 0.95 meaning that the top 95% of the probability mass is considered in the distribution.
# The repetition_penalty argument makes repeating output less likely by applying a penalty of 1.15.
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_length=2048,
    temperature=0.7,
    top_p=0.95,
    repetition_penalty=1.15
    # device=0 if torch.cuda.is_available() else -1  # Use GPU if available, else CPU. This line is currently commented out.
)

# Creates a HuggingFacePipeline object with the created pipeline.
llm = HuggingFacePipeline(pipeline=pipe)

# Sets the model name for the HuggingFace embeddings.
model_name = "intfloat/e5-large-v2"
# Creates a HuggingFaceEmbeddings object with the specified model name.
hf = HuggingFaceEmbeddings(model_name=model_name)


No sentence-transformers model found with name /root/.cache/torch/sentence_transformers/intfloat_e5-large-v2. Creating a new one with MEAN pooling.


# create the DB

In [ ]:
# Sets the directory where the embeddings will be stored.
persist_directory = 'db'

# Sets the HuggingFaceEmbeddings object as the embedding to use.
embedding = hf

# Uses the Chroma module to convert the texts into embeddings using the specified HuggingFace embeddings.
# The resulting embeddings are stored in the persist_directory.
vectordb = Chroma.from_documents(documents=texts,
                                 embedding=embedding,
                                 persist_directory=persist_directory)


In [ ]:

# Persists the generated embeddings database to disk.
vectordb.persist()

# Releases the memory held by the vectordb object by setting it to None.
vectordb = None


In [ ]:

# Re-initializes the Chroma object from the persisted directory with the specified HuggingFace embeddings.
vectordb = Chroma(persist_directory=persist_directory, embedding_function=embedding)


# Make a retriever

In [ ]:
# Initializes a retriever object from the vectordb with a specified number of search results (k=3).
# The vectordb.as_retriever() function is likely converting the Chroma object, which contains vectorized representations of text, into a retriever object.
# The retriever can be used to find the most similar vectors in the database given a query vector.
# The search_kwargs={"k": 3} argument suggests that the retriever will return the top 3 most similar vectors for each query.
retriever = vectordb.as_retriever(search_kwargs={"k": 3})


# Make a chain

In [ ]:
# Initializes a RetrievalQA object with the specified HuggingFacePipeline (llm), retriever, and chain type.
# The return_source_documents parameter set to True means the original source documents will be included in the returned results.
qa_chain = RetrievalQA.from_chain_type(llm=llm,
                                       chain_type="stuff",
                                       retriever=retriever,
                                       return_source_documents=True)


In [ ]:

 # This function processes the response from a Language Model (llm) and prints the result and sources.
def process_llm_response(llm_response):
    # Prints the 'result' field from the response, which likely contains the answer from the language model.
    print(llm_response['result'])

    print('\n\nSources:')
    # Iterates over the 'source_documents' field in the response, which contains the documents where the answer was found.
    for source in llm_response["source_documents"]:
        # Prints the 'source' metadata from each document.
        print(source.metadata['source'])



In [ ]:
query = "Could you please enumerate the companies that have been highlighted for their potential stock growth"
llm_response = qa_chain(query)
process_llm_response(llm_response)

 The eight companies that have been highlighted for their potential stock growth are Arbor Metals Corp., Tesla Inc., Kiplin Metals Inc., Microsoft, NVIDIA, Shopify, Intuitive Surgical, MercadoLibre, and Growth stocks like these are ideal for investors seeking short-term gains and long-term resilience.


Sources:
stock_market_june_2023/stock_market.txt
stock_market_june_2023/Microsoft Corporation.txt
stock_market_june_2023/stock_market.txt


In [ ]:
query = "how many companies that have been highlighted for their potential stock growth"
llm_response = qa_chain(query)
process_llm_response(llm_response)

 I am sorry, I do not see any information regarding the number of companies that have been highlighted for their potential stock growth in the given text.


Sources:
stock_market_june_2023/stock_market.txt
stock_market_june_2023/stock_market.txt
stock_market_june_2023/Microsoft Corporation.txt


In [ ]:
query = "What are the top five companies and how much did their stock value increase?"
llm_response = qa_chain(query)
process_llm_response(llm_response)


 I'm sorry, but the information provided does not include the top five companies or their specific stock value increases.


Sources:
stock_market_june_2023/stock_market.txt
stock_market_june_2023/NVIDIA Corporation.txt
stock_market_june_2023/Arbor Metals Corp.txt


In [ ]:
query = "How much has Microsoft invested in OpenAI?"
llm_response = qa_chain(query)
process_llm_response(llm_response)


 According to the text, Microsoft has invested $1 billion in OpenAI.


Sources:
stock_market_june_2023/Microsoft Corporation.txt
stock_market_june_2023/Microsoft Corporation.txt
stock_market_june_2023/Microsoft Corporation.txt


In [ ]:
query = "What were the reasons behind Shopify's decision to lay off a portion of its workforce"
llm_response = qa_chain(query)
process_llm_response(llm_response)

 Based on the information provided, it is not explicitly stated why Shopify decided to lay off a portion of its workforce. However, the text mentions that the company implemented "significant changes" to adapt to the changing landscape, which included selling its logistics business to Flexport and reducing overhead costs. It can be inferred that the layoffs may have been a part of this effort to streamline operations and become more asset-light in the long run.


Sources:
stock_market_june_2023/Shopify.txt
stock_market_june_2023/Shopify.txt
stock_market_june_2023/Shopify.txt


In [ ]:
query = "How does NVIDIA's dominance in the GPU market contribute to its strategic position in the AI industry?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

 NVIDIA's dominance in the GPU market gives it a strong position in the AI industry because many AI algorithms require specialized hardware such as GPUs to train and run efficiently. Since NVIDIA is the leading manufacturer of GPUs, it has a large market share and can leverage this to provide the necessary hardware for AI companies, making it a key player in the industry. Additionally, since AI is expected to grow significantly in the coming years, the demand for GPUs will likely increase, further solidifying NVIDIA's position.


Sources:
stock_market_june_2023/NVIDIA Corporation.txt
stock_market_june_2023/NVIDIA Corporation.txt
stock_market_june_2023/NVIDIA Corporation.txt
